In [ ]:
# For tips on running notebooks in Google Colab, see
# https://docs.pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\| **Optimization** \|\| [Save &
Load Model](saveloadrun_tutorial.html)

Optimizing Model Parameters
===========================

Now that we have a model and data it\'s time to train, validate and test
our model by optimizing its parameters on our data. Training a model is
an iterative process; in each iteration the model makes a guess about
the output, calculates the error in its guess (*loss*), collects the
derivatives of the error with respect to its parameters (as we saw in
the [previous section](autograd_tutorial.html)), and **optimizes** these
parameters using gradient descent. For a more detailed walkthrough of
this process, check out this video on [backpropagation from
3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

Prerequisite Code
-----------------

We load the code from the previous sections on [Datasets &
DataLoaders](data_tutorial.html) and [Build
Model](buildmodel_tutorial.html).


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26.4M/26.4M [00:01<00:00, 13.3MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 212kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.94MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 19.7MB/s]


Hyperparameters
===============

Hyperparameters are adjustable parameters that let you control the model
optimization process. Different hyperparameter values can impact model
training and convergence rates ([read
more](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)
about hyperparameter tuning)

We define the following hyperparameters for training:

:   -   **Number of Epochs** - the number of times to iterate over the
        dataset
    -   **Batch Size** - the number of data samples propagated through
        the network before the parameters are updated
    -   **Learning Rate** - how much to update models parameters at each
        batch/epoch. Smaller values yield slow learning speed, while
        large values may result in unpredictable behavior during
        training.


In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

Optimization Loop
=================

Once we set our hyperparameters, we can then train and optimize our
model with an optimization loop. Each iteration of the optimization loop
is called an **epoch**.

Each epoch consists of two main parts:

:   -   **The Train Loop** - iterate over the training dataset and try
        to converge to optimal parameters.
    -   **The Validation/Test Loop** - iterate over the test dataset to
        check if model performance is improving.

Let\'s briefly familiarize ourselves with some of the concepts used in
the training loop. Jump ahead to see the
`full-impl-label`{.interpreted-text role="ref"} of the optimization
loop.

Loss Function
-------------

When presented with some training data, our untrained network is likely
not to give the correct answer. **Loss function** measures the degree of
dissimilarity of obtained result to the target value, and it is the loss
function that we want to minimize during training. To calculate the loss
we make a prediction using the inputs of our given data sample and
compare it against the true data label value.

Common loss functions include
[nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)
(Mean Square Error) for regression tasks, and
[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)
(Negative Log Likelihood) for classification.
[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)
combines `nn.LogSoftmax` and `nn.NLLLoss`.

We pass our model\'s output logits to `nn.CrossEntropyLoss`, which will
normalize the logits and compute the prediction error.


In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

Optimizer
=========

Optimization is the process of adjusting model parameters to reduce
model error in each training step. **Optimization algorithms** define
how this process is performed (in this example we use Stochastic
Gradient Descent). All optimization logic is encapsulated in the
`optimizer` object. Here, we use the SGD optimizer; additionally, there
are many [different
optimizers](https://pytorch.org/docs/stable/optim.html) available in
PyTorch such as ADAM and RMSProp, that work better for different kinds
of models and data.

We initialize the optimizer by registering the model\'s parameters that
need to be trained, and passing in the learning rate hyperparameter.


In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:

:   -   Call `optimizer.zero_grad()` to reset the gradients of model
        parameters. Gradients by default add up; to prevent
        double-counting, we explicitly zero them at each iteration.
    -   Backpropagate the prediction loss with a call to
        `loss.backward()`. PyTorch deposits the gradients of the loss
        w.r.t. each parameter.
    -   Once we have our gradients, we call `optimizer.step()` to adjust
        the parameters by the gradients collected in the backward pass.


Full Implementation {#full-impl-label}
===================

We define `train_loop` that loops over our optimization code, and
`test_loop` that evaluates the model\'s performance against our test
data.

# **对下面代码的一些解释：**
这段代码是 PyTorch 训练模型时的“标准起手式”。如果把训练模型比作**吃自助餐**，那么这段代码的逻辑就是：**分批次把菜送到你面前，并记下这是第几盘。**

下面是它的深度拆解：

---

### 1. 核心组件拆解

| 组件 | 现实类比 | 逻辑功能 |
| --- | --- | --- |
| **`dataloader`** | 后厨传送带 | 一个可迭代对象。它负责把原始数据（可能是几万张图片）进行打乱（shuffle）、分组（batching）和多线程加载。 |
| **`enumerate()`** | 计数器 | Python 内置函数。它会给迭代的对象加个序号，从 `0` 开始。 |
| **`batch`** | 批次序号 | 记录当前是第几个批次（第 0 批、第 1 批...），通常用于打印日志或调整学习率。 |
| **`(X, y)`** | 一份套餐 | **元组拆解（Unpacking）**。`X` 通常是特征数据（张量），`y` 是对应的标签（答案）。 |

---

### 2. 执行流程（每一轮循环在干什么？）

当你运行这行代码时，后台会发生以下“连环动作”：

1. **取样（Sampling）**：`dataloader` 根据设定的 `batch_size`（比如 32），从数据集里随机或按顺序抓出 32 个样本的索引。
2. **加载（Loading）**：根据索引去硬盘或内存里把这 32 组数据读出来。
3. **转换（Transforming）**：如果你定义了 `transform`（比如缩放图片、转为 Tensor），它会在此刻对这 32 组数据进行处理。
4. **打包（Batching）**：将 32 组独立的数据“粘”在一起，压成一个高维张量（Tensor）。例如  的形状会变成 `[32, channels, height, width]`。
5. **投递**：
* `enumerate` 扔出一个数字（如 `0`）。
* `dataloader` 扔出一对数据 `(X, y)`。
* 进入循环体，开始执行 `loss = model(X)` 等计算。



---
#**dataloader 扔出一对数据 (X, y)。这里扔出一对数据是把32组图片的数据都给扔出来吗？**
是的，你的理解非常到位！**它不是一个一个往外扔，而是把 32 组数据“打包”成一个整体，一次性扔给你。**

在深度学习中，我们管这个“整体”叫一个 **Batch（批次）**。

### 1. 它是如何“打包”的？

假设你处理的是彩色图片，每张图片的大小是 ：

* **单张图片的数据：** 形状是 `[3, 224, 224]`（3 是 RGB 通道）。
* **DataLoader 扔出的 ：** 形状会变成 `[32, 3, 224, 224]`。

这里的 **`32`** 就是第一个维度（Batch Size）。这就像是把 32 张照片叠在一起放进了一个盒子里，`X` 就是这个盒子。

同样的，****（标签）也不会是 32 个独立的数字，而是一个包含了 32 个数字的**向量（一维张量）**，形状是 `[32]`。

---

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to
`train_loop` and `test_loop`. Feel free to increase the number of epochs
to track the model\'s improving performance.


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307117  [   64/60000]
loss: 2.293414  [ 6464/60000]
loss: 2.275460  [12864/60000]
loss: 2.272308  [19264/60000]
loss: 2.253887  [25664/60000]
loss: 2.219176  [32064/60000]
loss: 2.240055  [38464/60000]
loss: 2.201148  [44864/60000]
loss: 2.203244  [51264/60000]
loss: 2.176103  [57664/60000]
Test Error: 
 Accuracy: 30.0%, Avg loss: 2.169092 

Epoch 2
-------------------------------
loss: 2.177977  [   64/60000]
loss: 2.168671  [ 6464/60000]
loss: 2.117861  [12864/60000]
loss: 2.140019  [19264/60000]
loss: 2.085822  [25664/60000]
loss: 2.023813  [32064/60000]
loss: 2.067814  [38464/60000]
loss: 1.985826  [44864/60000]
loss: 1.998083  [51264/60000]
loss: 1.935785  [57664/60000]
Test Error: 
 Accuracy: 49.3%, Avg loss: 1.928934 

Epoch 3
-------------------------------
loss: 1.959051  [   64/60000]
loss: 1.932425  [ 6464/60000]
loss: 1.824588  [12864/60000]
loss: 1.868521  [19264/60000]
loss: 1.758034  [25664/60000]
loss: 1.703062  [32064/600

Further Reading
===============

-   [Loss
    Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
-   [torch.optim](https://pytorch.org/docs/stable/optim.html)
-   [Warmstart Training a
    Model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)
